# 개요
1. 제주 전체의 매출과 이익 개선을 위해 부름 지역을 확대하고 부름 편도를 우선적으로 오픈한다
- 목표 : 기존의 전체 건수에서 왕복 예약 비중은 유지하면서, 부름으로 인한 건수를 3% 증가 시킨다

# 살펴보기
1. 제주지역의 부름 운영 시작이후 왕복 예약 비중은 유지되고 있는지 ?   
1-1 전체 건수는 증가 하였는지 ?
- 건수
- 매출
- 이익(GP)
3. 지역별 현황은 어떠한지
- 제주시
- 서귀포시
4. 더 개선할 수 있는 부분은 어디인지

In [ ]:
from google.colab import output
output.clear()

---

# 필요한 라이브러리 준비 ⏳

In [ ]:
import pandas as pd
import numpy as np
from plotnine import *
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import gspread
from google.auth import default
creds, _ = default()
from gspread_dataframe import get_as_dataframe, set_with_dataframe

from google.cloud import bigquery
from oauth2client.client import GoogleCredentials
from sklearn.preprocessing import MinMaxScaler

gc = gspread.authorize(creds)

### 필요한 데이터를 빅쿼리에 연동하여 가져옴
* 부름 운영 전후 4주간의 실적 비교

In [ ]:
query = """
  SELECT
      CASE WHEN date >= '2023-11-06' AND date <= '2023-12-07' THEN 'As-is'
           WHEN date >= '2023-12-08' AND date <= '2024-01-08' THEN 'To-be' END as date
      , r.way as way
      , r.region2

      , count(r.reservation_id) as nuse -- 예약건수
      , sum(r.utime) as utime -- 이용시간

      , sum(revenue) as revenue -- 회계매출
      , sum(_rev_d2d) as _rev_d2d -- 부름매출(부름요금 + 편도요금)

      , sum(contribution_margin) as margin -- 공헌이익

      ,sum(transport_cost_d2d) as d2d_cost -- 부름배반비용

  FROM `socar-data.soda_store.reservation_v2` r
  LEFT JOIN socar-data.tianjin_replica.reservation_info i ON r.reservation_id = i.id
  LEFT JOIN socar-data.tianjin_replica.reservation_dtod_info d ON r.reservation_id = d.reservation_id
  LEFT JOIN `tianjin_replica.car_info` c ON r.car_id = c.id
  LEFT JOIN `tianjin_replica.car_class` cl ON c.class_id = cl.id
  WHERE 1=1
  AND date >= '2023-11-06' AND date <= '2024-01-08'
  AND r.member_imaginary IN (0,9)
  AND region1 = '제주특별자치도'
  GROUP BY date, way, region2
  ORDER BY date, way, region2
  """

df = pd.io.gbq.read_gbq(
    query=query,
    project_id="socar-data"
)

In [ ]:
df.info()

In [ ]:
df['region2'].unique()

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
pd.set_option('display.float_format', '{:1.0f}'.format)

In [ ]:
df['revenue_nuse'] = df['revenue'] / df['nuse'].replace({0: 1})  # 0인 경우를 방지
df['utime_nuse'] = df['utime'] / df['nuse'].replace({0: 1})  # 0인 경우를 방지
df['margin_nuse'] = df['margin'] / df['nuse'].replace({0: 1})  # 0인 경우를 방지
df['d2d_cost_nuse'] = df['d2d_cost'] / df['nuse'].replace({0: 1})  # 0인 경우를 방지

---
# 1. 제주지역의 부름 운영 시작이후 왕복 예약 비중은 유지되고 있는지 ?   
1-1 전체 건수는 증가 하였는지 ?
- 건수
- 매출
- 이익(GP)
---

## 📈 *건수, 시간의  예약타입별 구성비*

✔ 예상과 다르게 건수가 확대 되지 않았다. (구성비는 비슷하다)

In [ ]:
df.info()

In [ ]:
df_grouped = df.groupby(['way', 'date'])[['nuse', 'utime']].sum().reset_index()

# 'As-is'와 'To-be' 데이터를 wide format으로 변환
df_pivot = df_grouped.pivot(index='way', columns='date', values=['nuse', 'utime'])

# 컬럼 이름 재정의
df_pivot.columns = [f'{i}_{j}' for i, j in df_pivot.columns]

# 증감률 계산
df_pivot['nuse_growth'] = (df_pivot['nuse_To-be'] - df_pivot['nuse_As-is']) / df_pivot['nuse_As-is'] * 100
df_pivot['utime_growth'] = (df_pivot['utime_To-be'] - df_pivot['utime_As-is']) / df_pivot['utime_As-is'] * 100

# 결과 데이터프레임 재구성
result_df = df_pivot.reset_index()
result_df = result_df[['way', 'nuse_As-is', 'nuse_To-be', 'nuse_growth', 'utime_As-is', 'utime_To-be', 'utime_growth']]
result_df

In [ ]:
result_df.info()

In [ ]:
# 단계 1: date별로 nuse의 합계를 계산
date_totals = df.groupby('date')['nuse'].sum().reset_index()

# 단계 2: date와 way별로 nuse의 합계를 계산 후, date의 전체 nuse 값에 대한 구성비 계산
summary = df.groupby(['date', 'way'])['nuse'].sum().reset_index()
summary = summary.merge(date_totals, on='date', suffixes=('', '_total'))
summary['proportion'] = (summary['nuse'] / summary['nuse_total']) * 100

In [ ]:
summary.info()

In [ ]:
# 'As-is'와 'To-be' 데이터 분리
asis_data = summary[summary['date'] == 'As-is']
tobe_data = summary[summary['date'] == 'To-be']

# 'As-is' 데이터를 음수로 변환하여 'To-be'와 대칭되게 만듦
asis_data['nuse'] = asis_data['nuse'] * -1

# 피라미드 차트 생성
fig = go.Figure()

# 'As-is' 데이터 추가 (음수값 사용)
fig.add_trace(go.Bar(
    y=asis_data['way'],
    x=asis_data['nuse'],
    name='As-is',
    orientation='h',
    # marker=dict(color='rgba(55, 128, 191, 0.7)'),
    text=-asis_data['nuse']  # 텍스트는 양수로 표시
))

# 'To-be' 데이터 추가
fig.add_trace(go.Bar(
    y=tobe_data['way'],
    x=tobe_data['nuse'],
    name='To-be',
    orientation='h',
    # marker=dict(color='rgba(255, 153, 51, 0.7)'),
    text=tobe_data['nuse']
))

# 레이아웃 업데이트
fig.update_layout(
    title='Date별 예약타입별 nuse (피라미드 차트)',
    title_font_size=22,
    barmode='overlay',  # 바가 겹치지 않도록 설정
    bargap=0.1,  # 바 간의 간격
    xaxis=dict(
        title='nuse 건수',
        title_font_size=14,
        tickvals=[i for i in range(-max(summary['nuse']), max(summary['nuse'])+1, int(max(summary['nuse'])/5))],
        ticktext=[str(abs(i)) for i in range(-max(summary['nuse']), max(summary['nuse'])+1, int(max(summary['nuse'])/5))]
    ),
    yaxis=dict(
        title='예약타입',
        title_font_size=14
    )
)

fig.show()

In [ ]:
pd.set_option('display.float_format', '{:1.0f}'.format)

In [ ]:
import plotly.graph_objects as go

# 'As-is'와 'To-be' 데이터에 대해 각각 nuse의 합계를 구합니다.
total_nuse_asis = summary[summary['date'] == 'As-is']['nuse'].sum()
total_nuse_tobe = summary[summary['date'] == 'To-be']['nuse'].sum()

# 각 'way'별 'nuse' 합계의 전체 합계 대비 비율을 계산합니다.
asis_data = summary[summary['date'] == 'As-is'].copy()
tobe_data = summary[summary['date'] == 'To-be'].copy()

asis_data['proportion'] = asis_data['nuse'] / total_nuse_asis
tobe_data['proportion'] = tobe_data['nuse'] / total_nuse_tobe

# 'As-is' 데이터의 비율을 음수로 변환하여 대칭 효과를 생성합니다.
asis_data['proportion'] *= -1

# 피라미드 차트 생성
fig = go.Figure()

# 'As-is' 데이터 추가 (음수 비율 사용)
fig.add_trace(go.Bar(
    y=asis_data['way'],
    x=asis_data['proportion'],
    name='As-is',
    orientation='h',
    # 비율을 퍼센트로 변환하여 정수 부분만 표시합니다.
    text=asis_data['proportion'].apply(lambda x: f"{int(abs(x) * 100)}%")
))

# 'To-be' 데이터 추가 (양수 비율 사용)
fig.add_trace(go.Bar(
    y=tobe_data['way'],
    x=tobe_data['proportion'],
    name='To-be',
    orientation='h',
    # 비율을 퍼센트로 변환하여 정수 부분만 표시합니다.
    text=tobe_data['proportion'].apply(lambda x: f"{int(x * 100)}%")
))

# 레이아웃 업데이트
fig.update_layout(
    title='Date별 예약타입별 nuse 비율 (피라미드 차트)',
    title_font_size=22,
    barmode='overlay',
    bargap=0.1,
    xaxis=dict(
        title='비율',
        title_font_size=14,
        tickformat=',.0%'
    ),
    yaxis=dict(
        title='예약타입',
        title_font_size=14
    )
)

fig.show()

In [ ]:
# 'As-is'와 'To-be' 데이터 분리
asis_data = summary[summary['date'] == 'As-is']
tobe_data = summary[summary['date'] == 'To-be']

# 도넛 차트 생성
fig = go.Figure()

fig.add_trace(go.Pie(labels=asis_data['way'],
                     values=asis_data['proportion'],
                     name='As-is',
                     hole=0.4,  # 중앙 구멍의 크기 설정
                     domain={'x': [0, 0.48]},  # 'As-is'를 도넛 차트 왼쪽에 배치
                     hoverinfo='label+percent+name'))

fig.add_trace(go.Pie(labels=tobe_data['way'],
                     values=tobe_data['proportion'],
                     name='To-be',
                     hole=0.4,
                     domain={'x': [0.52, 1.0]},  # 'To-be'를 도넛 차트 오른쪽에 배치
                     hoverinfo='label+percent+name'))

# 레이아웃 업데이트 (제목 위치 조정)
fig.update_layout(
    title_text='As-is vs To-be 예약타입별 nuse의 구성비',
    # 차트 가운데 제목 대신에 외부에 표시
    annotations=[dict(text='As-is', x=0.1, y=1.15, font_size=16, showarrow=False),
                 dict(text='To-be', x=0.9, y=1.15, font_size=16, showarrow=False)],
    showlegend=True)

fig.show()

In [ ]:
# 단계 1: date별로 nuse의 합계를 계산
date_totals = df.groupby('date')['utime'].sum().reset_index()

# 단계 2: date와 way별로 nuse의 합계를 계산 후, date의 전체 nuse 값에 대한 구성비 계산
summary = df.groupby(['date', 'way'])['utime'].sum().reset_index()
summary = summary.merge(date_totals, on='date', suffixes=('', '_total'))

# 바 차트로 결과 시각화
fig = px.bar(summary, x='way', y='utime', color='date',
             title='Date별 예약타입별 utime',
             barmode='group',
             text='utime',
             labels={'utime': '이용시간', 'way': '예약타입', 'date': 'Date'})

fig.update_traces(texttemplate='%{text:1.0f}', textposition='outside')
fig.show()

In [ ]:
# 데이터 준비: 'As-is'와 'To-be'의 구성비 계산
summary['proportion'] = summary['utime'] / summary['utime_total'] * 100

# 'As-is' 데이터를 음수로 변환하여 대칭 효과 생성
asis_data = summary[summary['date'] == 'As-is'].copy()
tobe_data = summary[summary['date'] == 'To-be'].copy()

asis_data['proportion'] *= -1

# 피라미드 차트 생성을 위한 준비
fig = go.Figure()

# 'As-is' 데이터 추가 (음수값 사용)
fig.add_trace(go.Bar(
    y=asis_data['way'],
    x=asis_data['proportion'],
    name='As-is',
    orientation='h',
    text=asis_data['proportion'].apply(lambda x: f"{abs(x):.2f}%"),  # 양수 비율로 텍스트 표시
    # marker_color='rgba(55, 128, 191, 0.7)'
))

# 'To-be' 데이터 추가 (양수값 사용)
fig.add_trace(go.Bar(
    y=tobe_data['way'],
    x=tobe_data['proportion'],
    name='To-be',
    orientation='h',
    text=tobe_data['proportion'].apply(lambda x: f"{x:.2f}%"),
    # marker_color='rgba(255, 153, 51, 0.7)'
))

# 레이아웃 업데이트
fig.update_layout(
    title='Date별 예약타입별 utime 구성비 (피라미드 차트)',
    title_font_size=22,
    barmode='overlay',  # 바가 겹치지 않도록 설정
    bargap=0.1,  # 바 간의 간격
    xaxis=dict(
        title='구성비 (%)',
        title_font_size=14,
        tickformat=',.2f'  # 비율 형식으로 축 표시
    ),
    yaxis=dict(
        title='예약타입',
        title_font_size=14,
        categoryorder='total ascending'  # 'way'를 기준으로 정렬
    )
)

fig.show()

## 📈 *지역별 건수, 시간의 예약타입별 구성비*

✔ 예상과 다르게 건수가 확대 되지 않았다. (구성비는 비슷하다)

In [ ]:
df_grouped = df.groupby(['region2', 'way', 'date'])[['nuse', 'utime']].sum().reset_index()

df_pivot = df_grouped.pivot_table(index=['region2', 'way'], columns='date', values=['nuse', 'utime'])

# 컬럼 이름 재정의
df_pivot.columns = [f'{i}_{j}' for i, j in df_pivot.columns]

df_pivot['nuse_growth'] = (df_pivot['nuse_To-be'] - df_pivot['nuse_As-is']) / df_pivot['nuse_As-is'].replace(0, 1) * 100
df_pivot['utime_growth'] = (df_pivot['utime_To-be'] - df_pivot['utime_As-is']) / df_pivot['utime_As-is'].replace(0, 1) * 100

result_df = df_pivot.reset_index()
result_df = result_df[['region2', 'way', 'nuse_As-is', 'nuse_To-be', 'nuse_growth', 'utime_As-is', 'utime_To-be', 'utime_growth']]
print(result_df)
result_df

In [ ]:
df.info()

In [ ]:
# 단계 1: date와 region2별로 nuse의 합계를 계산
date_region2_totals = df.groupby(['date', 'region2'])['nuse'].sum().reset_index()

# 단계 2: date, region2, way별로 nuse의 합계를 계산 후, date와 region2의 조합별 전체 nuse 값에 대한 구성비 계산
summary = df.groupby(['date', 'region2', 'way'])['nuse'].sum().reset_index()
summary = summary.merge(date_region2_totals, on=['date', 'region2'], suffixes=('', '_total'))

# 바 차트로 결과 시각화
fig = px.bar(summary, x='way', y='nuse', color='date',
             facet_col='region2', # region2별로 차트 분할
             title='Date 및 Region2별 예약타입별 nuse',
             barmode='group',
             text='nuse',
             labels={'nuse': '건수', 'way': '예약타입', 'date': 'Date', 'region2': 'Region2'})

fig.update_traces(texttemplate='%{text:1.0f}', textposition='outside')
fig.show()

In [ ]:
# 단계 1: date와 region2별로 nuse의 합계를 계산
date_region2_totals = df.groupby(['date', 'region2'])['nuse'].sum().reset_index()

# 단계 2: date, region2, way별로 nuse의 합계를 계산 후, date와 region2의 조합별 전체 nuse 값에 대한 구성비 계산
summary = df.groupby(['date', 'region2', 'way'])['nuse'].sum().reset_index()
summary = summary.merge(date_region2_totals, on=['date', 'region2'], suffixes=('', '_total'))
summary['proportion'] = (summary['nuse'] / summary['nuse_total']) * 100

# 바 차트로 결과 시각화
fig = px.bar(summary, x='way', y='proportion', color='date',
             facet_col='region2', # region2별로 차트 분할
             title='Date 및 Region2별 예약타입별 nuse의 구성비',
             barmode='group',
             text='proportion',
             labels={'proportion': '구성비 (%)', 'way': '예약타입', 'date': 'Date', 'region2': 'Region2'})

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.show()

In [ ]:
# 정규화를 위한 Scaler 초기화
scaler = MinMaxScaler()

# 'utime'에 대한 전체 데이터를 정규화
summary['nuse_normalized'] = scaler.fit_transform(summary[['nuse']])

# 'date'와 'region2'별 전체 'utime' 값에 대한 구성비 계산을 정규화된 값으로 수정
summary['nuse'] = summary['nuse_normalized']

# 바 차트로 결과 시각화
fig = px.bar(summary, x='way', y='nuse', color='date',
             facet_col='region2',  # region2별로 차트 분할
             title='Date 및 Region2별 예약타입별 nuse (정규화)',
             barmode='group',
             text='nuse',
             labels={'nuse': '이용건수 (정규화)', 'way': '예약타입', 'date': 'Date', 'region2': 'Region2'})

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.show()

In [ ]:
# 단계 1: date와 region2별로 nuse의 합계를 계산
date_region2_totals = df.groupby(['date', 'region2'])['utime'].sum().reset_index()

# 단계 2: date, region2, way별로 nuse의 합계를 계산 후, date와 region2의 조합별 전체 nuse 값에 대한 구성비 계산
summary = df.groupby(['date', 'region2', 'way'])['utime'].sum().reset_index()
summary = summary.merge(date_region2_totals, on=['date', 'region2'], suffixes=('', '_total'))

# 바 차트로 결과 시각화
fig = px.bar(summary, x='way', y='utime', color='date',
             facet_col='region2', # region2별로 차트 분할
             title='Date 및 Region2별 예약타입별 utime',
             barmode='group',
             text='utime',
             labels={'utime': '이용시간', 'way': '예약타입', 'date': 'Date', 'region2': 'Region2'})

fig.update_traces(texttemplate='%{text:1.0f}', textposition='outside')
fig.show()

In [ ]:
# 단계 1: date와 region2별로 nuse의 합계를 계산
date_region2_totals = df.groupby(['date', 'region2'])['utime'].sum().reset_index()

# 단계 2: date, region2, way별로 nuse의 합계를 계산 후, date와 region2의 조합별 전체 nuse 값에 대한 구성비 계산
summary = df.groupby(['date', 'region2', 'way'])['utime'].sum().reset_index()
summary = summary.merge(date_region2_totals, on=['date', 'region2'], suffixes=('', '_total'))
summary['proportion'] = (summary['utime'] / summary['utime_total']) * 100

# 바 차트로 결과 시각화
fig = px.bar(summary, x='way', y='proportion', color='date',
             facet_col='region2', # region2별로 차트 분할
             title='Date 및 Region2별 예약타입별 utime의 구성비',
             barmode='group',
             text='proportion',
             labels={'proportion': '구성비 (%)', 'way': '예약타입', 'date': 'Date', 'region2': 'Region2'})

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.show()

In [ ]:
# 정규화를 위한 Scaler 초기화
scaler = MinMaxScaler()

# 'utime'에 대한 전체 데이터를 정규화
summary['utime_normalized'] = scaler.fit_transform(summary[['utime']])

# 'date'와 'region2'별 전체 'utime' 값에 대한 구성비 계산을 정규화된 값으로 수정
summary['utime'] = summary['utime_normalized']

# 바 차트로 결과 시각화
fig = px.bar(summary, x='way', y='utime', color='date',
             facet_col='region2',  # region2별로 차트 분할
             title='Date 및 Region2별 예약타입별 utime (정규화)',
             barmode='group',
             text='utime',
             labels={'utime': '이용시간 (정규화)', 'way': '예약타입', 'date': 'Date', 'region2': 'Region2'})

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.show()

## 📈 *매출과 공헌이익의  예약타입별 구성비*

✔ 예상과 다르게 건수가 확대 되지 않았다. (구성비는 비슷하다)

In [ ]:
df.info()

In [ ]:
df_grouped = df.groupby(['way', 'date'])[['revenue', 'margin']].sum().reset_index()

# 'As-is'와 'To-be' 데이터를 wide format으로 변환
df_pivot = df_grouped.pivot(index='way', columns='date', values=['revenue', 'margin'])

# 컬럼 이름 재정의
df_pivot.columns = [f'{i}_{j}' for i, j in df_pivot.columns]

# 증감률 계산
df_pivot['revenue_growth'] = (df_pivot['revenue_To-be'] - df_pivot['revenue_As-is']) / df_pivot['revenue_As-is'] * 100
df_pivot['margin_growth'] = (df_pivot['margin_To-be'] - df_pivot['margin_As-is']) / df_pivot['margin_As-is'] * 100

# 결과 데이터프레임 재구성
result_df = df_pivot.reset_index()
result_df = result_df[['way', 'revenue_As-is', 'revenue_To-be', 'revenue_growth', 'margin_As-is', 'margin_To-be', 'margin_growth']]
result_df

In [ ]:
# 단계 1: date별로 nuse의 합계를 계산
date_totals = df.groupby('date')['revenue'].sum().reset_index()

# 단계 2: date와 way별로 nuse의 합계를 계산 후, date의 전체 nuse 값에 대한 구성비 계산
summary = df.groupby(['date', 'way'])['revenue'].sum().reset_index()
summary = summary.merge(date_totals, on='date', suffixes=('', '_total'))

# 바 차트로 결과 시각화
fig = px.bar(summary, x='way', y='revenue', color='date',
             title='Date별 예약타입별 revenue',
             barmode='group',
             text='revenue',
             labels={'revenue': '매출', 'way': '예약타입', 'date': 'Date'})

fig.update_traces(texttemplate='%{text:1.0f}', textposition='outside')
fig.show()

In [ ]:
# 단계 1: date별로 nuse의 합계를 계산
date_totals = df.groupby('date')['margin'].sum().reset_index()

# 단계 2: date와 way별로 nuse의 합계를 계산 후, date의 전체 nuse 값에 대한 구성비 계산
summary = df.groupby(['date', 'way'])['margin'].sum().reset_index()
summary = summary.merge(date_totals, on='date', suffixes=('', '_total'))

# 바 차트로 결과 시각화
fig = px.bar(summary, x='way', y='margin', color='date',
             title='Date별 예약타입별 margin',
             barmode='group',
             text='margin',
             labels={'revmarginenue': '공헌이익', 'way': '예약타입', 'date': 'Date'})

fig.update_traces(texttemplate='%{text:1.0f}', textposition='outside')
fig.show()

In [ ]:
# 'revenue'와 'margin'에 대한 date별 총합 계산
date_totals_revenue = df.groupby('date')['revenue'].sum().rename('revenue_total').reset_index()
date_totals_margin = df.groupby('date')['margin'].sum().rename('margin_total').reset_index()

# date와 way별로 'revenue'와 'margin'의 합계 계산
summary_revenue = df.groupby(['date', 'way'])['revenue'].sum().reset_index()
summary_margin = df.groupby(['date', 'way'])['margin'].sum().reset_index()

# date별 총합과 병합하여 구성비 계산을 위한 준비
summary_revenue = summary_revenue.merge(date_totals_revenue, on='date')
summary_margin = summary_margin.merge(date_totals_margin, on='date')

# 구성비 계산
summary_revenue['revenue_proportion'] = (summary_revenue['revenue'] / summary_revenue['revenue_total']) * 100
summary_margin['margin_proportion'] = (summary_margin['margin'] / summary_margin['margin_total']) * 100

# 결과 데이터프레임 준비: 'revenue'와 'margin' 정보를 합치기
summary = pd.merge(summary_revenue, summary_margin[['date', 'way', 'margin', 'margin_proportion']], on=['date', 'way'])

# 필요한 컬럼만 선택하여 최종 데이터프레임 구성
result_df = summary[['date', 'way', 'revenue', 'revenue_proportion', 'margin', 'margin_proportion']]

In [ ]:
result_df.info()

In [ ]:
from plotly.subplots import make_subplots
# 두 개의 병렬 차트 생성을 위한 subplot 설정
fig = make_subplots(rows=1, cols=2, subplot_titles=('Revenue Proportion', 'Margin Proportion'))

# Revenue Proportion 차트 추가
for date in result_df['date'].unique():
    df_filtered = result_df[result_df['date'] == date]
    fig.add_trace(
        go.Bar(x=df_filtered['way'], y=df_filtered['revenue_proportion'], name=f'Revenue {date}', text=df_filtered['revenue_proportion']),
        row=1, col=1
    )

# Margin Proportion 차트 추가
for date in result_df['date'].unique():
    df_filtered = result_df[result_df['date'] == date]
    fig.add_trace(
        go.Bar(x=df_filtered['way'], y=df_filtered['margin_proportion'], name=f'Margin {date}', text=df_filtered['margin_proportion']),
        row=1, col=2
    )

# 레이아웃 업데이트
fig.update_layout(height=600, width=1200, title_text="Revenue and Margin by Date and Way")
fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.show()

In [ ]:
# 정규화를 위한 Scaler 초기화
scaler_revenue = MinMaxScaler()
scaler_margin = MinMaxScaler()

# 'revenue'와 'margin'의 정규화
summary['revenue_normalized'] = scaler_revenue.fit_transform(summary[['revenue']])
summary['margin_normalized'] = scaler_margin.fit_transform(summary[['margin']])

In [ ]:
# 두 개의 병렬 차트 생성을 위한 subplot 설정
fig = make_subplots(rows=1, cols=2, subplot_titles=('revenue_normalized', 'margin_normalized'))

# 정규화된 Revenue 차트 추가
for date in summary['date'].unique():
    df_filtered = summary[summary['date'] == date]
    fig.add_trace(
        go.Bar(x=df_filtered['way'], y=df_filtered['revenue_normalized'], name=f'Revenue {date}', text=df_filtered['revenue_normalized']),
        row=1, col=1
    )

# 정규화된 Margin 차트 추가
for date in summary['date'].unique():
    df_filtered = summary[summary['date'] == date]
    fig.add_trace(
        go.Bar(x=df_filtered['way'], y=df_filtered['margin_normalized'], name=f'Margin {date}', text=df_filtered['margin_normalized']),
        row=1, col=2
    )

# 레이아웃 업데이트
fig.update_layout(height=600, width=1200, title_text="정규화된 Revenue and Margin by Date and Way")
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.show()

## 📈 *지역별 매출과 공헌이익의  예약타입별 구성비*

✔ 예상과 다르게 건수가 확대 되지 않았다. (구성비는 비슷하다)

In [ ]:
df_grouped = df.groupby(['region2', 'way', 'date'])[['revenue', 'margin']].sum().reset_index()

df_pivot = df_grouped.pivot_table(index=['region2', 'way'], columns='date', values=['revenue', 'margin'])

# 컬럼 이름 재정의
df_pivot.columns = [f'{i}_{j}' for i, j in df_pivot.columns]

df_pivot['revenue_growth'] = (df_pivot['revenue_To-be'] - df_pivot['revenue_As-is']) / df_pivot['revenue_As-is'].replace(0, 1) * 100
df_pivot['margin_growth'] = (df_pivot['margin_To-be'] - df_pivot['margin_As-is']) / df_pivot['margin_As-is'].replace(0, 1) * 100

result_df = df_pivot.reset_index()
result_df = result_df[['region2', 'way', 'revenue_As-is', 'revenue_To-be', 'revenue_growth', 'margin_As-is', 'margin_To-be', 'margin_growth']]
print(result_df)
result_df

In [ ]:
# 단계 1: date와 region2별로 nuse의 합계를 계산
date_region2_totals = df.groupby(['date', 'region2'])['revenue'].sum().reset_index()

# 단계 2: date, region2, way별로 nuse의 합계를 계산 후, date와 region2의 조합별 전체 nuse 값에 대한 구성비 계산
summary = df.groupby(['date', 'region2', 'way'])['revenue'].sum().reset_index()
summary = summary.merge(date_region2_totals, on=['date', 'region2'], suffixes=('', '_total'))

# 바 차트로 결과 시각화
fig = px.bar(summary, x='way', y='revenue', color='date',
             facet_col='region2', # region2별로 차트 분할
             title='Date 및 Region2별 예약타입별 revenue',
             barmode='group',
             text='revenue',
             labels={'revenue': '매출', 'way': '예약타입', 'date': 'Date', 'region2': 'Region2'})

fig.update_traces(texttemplate='%{text:1.0f}', textposition='outside')
fig.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# 예제 데이터 프레임 'df'가 있다고 가정합니다.

# 단계 1: date와 region2별로 revenue의 합계를 계산
date_region2_totals = df.groupby(['date', 'region2'])['revenue'].sum().reset_index()

# 단계 2: date, region2, way별로 revenue의 합계를 계산 후, date와 region2의 조합별 전체 revenue 값에 대한 구성비 계산
summary = df.groupby(['date', 'region2', 'way'])['revenue'].sum().reset_index()
summary = summary.merge(date_region2_totals, on=['date', 'region2'], suffixes=('', '_total'))

# revenue에 대한 정규화를 진행
scaler = MinMaxScaler()
summary['revenue_normalized'] = scaler.fit_transform(summary[['revenue']])

# 바 차트로 정규화된 결과 시각화
fig = px.bar(summary, x='way', y='revenue_normalized', color='date',
             facet_col='region2',  # region2별로 차트 분할
             title='Date 및 Region2별 예약타입별 정규화된 revenue',
             barmode='group',
             text='revenue_normalized',
             labels={'revenue_normalized': '정규화된 매출', 'way': '예약타입', 'date': 'Date', 'region2': 'Region2'})

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')  # 소수점 두 자리로 표시
fig.show()

In [ ]:
# 단계 1: date와 region2별로 nuse의 합계를 계산
date_region2_totals = df.groupby(['date', 'region2'])['margin'].sum().reset_index()

# 단계 2: date, region2, way별로 nuse의 합계를 계산 후, date와 region2의 조합별 전체 nuse 값에 대한 구성비 계산
summary = df.groupby(['date', 'region2', 'way'])['margin'].sum().reset_index()
summary = summary.merge(date_region2_totals, on=['date', 'region2'], suffixes=('', '_total'))

# 바 차트로 결과 시각화
fig = px.bar(summary, x='way', y='margin', color='date',
             facet_col='region2', # region2별로 차트 분할
             title='Date 및 Region2별 예약타입별 margin',
             barmode='group',
             text='margin',
             labels={'margin': '공헌이익', 'way': '예약타입', 'date': 'Date', 'region2': 'Region2'})

fig.update_traces(texttemplate='%{text:1.0f}', textposition='outside')
fig.show()

In [ ]:
# 단계 1: date와 region2별로 margin의 합계를 계산
date_region2_totals = df.groupby(['date', 'region2'])['margin'].sum().reset_index()

# 단계 2: date, region2, way별로 margin의 합계를 계산 후, date와 region2의 조합별 전체 margin 값에 대한 구성비 계산
summary = df.groupby(['date', 'region2', 'way'])['margin'].sum().reset_index()
summary = summary.merge(date_region2_totals, on=['date', 'region2'], suffixes=('', '_total'))

# margin에 대한 정규화를 진행
scaler = MinMaxScaler()
summary['margin_normalized'] = scaler.fit_transform(summary[['margin']])

# 바 차트로 정규화된 결과 시각화
fig = px.bar(summary, x='way', y='margin_normalized', color='date',
             facet_col='region2',  # region2별로 차트 분할
             title='Date 및 Region2별 예약타입별 정규화된 공헌이익',
             barmode='group',
             text='margin_normalized',
             labels={'margin_normalized': '정규화된 공헌이익', 'way': '예약타입', 'date': 'Date', 'region2': 'Region2'})

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')  # 소수점 두 자리로 표시
fig.show()

## 📈 *합계가 아닌 건당 지표를 보는 것이, 타당하다*
- 건당 매출
- 건당 이용시간
- 건당 공헌이익

✔ 건수나 시간이 크게 늘어나지 않았지만
1. 건당 지표들이 크게 개선되었음

부름 예약을 확대할 수 있다면, 전체 매출과 이익을 개선하는게 큰 효과가 있을 것으로 보여짐

In [ ]:
# Step 2: date 및 date와 way별로 revenue_nuse의 평균 계산
summary = df.groupby(['date', 'way'])['revenue_nuse'].mean().reset_index()
date_totals = df.groupby('date')['revenue_nuse'].mean().reset_index()
summary = summary.merge(date_totals, on='date', suffixes=('', '_total'))

df_grouped = df.groupby(['way', 'date'])['revenue_nuse'].mean().reset_index()

# 'As-is'와 'To-be' 데이터를 wide format으로 변환
df_pivot = df_grouped.pivot(index='way', columns='date', values=['revenue_nuse'])

# 컬럼 이름 재정의
df_pivot.columns = [f'{i}_{j}' for i, j in df_pivot.columns]

# 증감률 계산
df_pivot['revenue_nuse_growth'] = (df_pivot['revenue_nuse_To-be'] - df_pivot['revenue_nuse_As-is']) / df_pivot['revenue_nuse_As-is'] * 100

# 결과 데이터프레임 재구성
result_df = df_pivot.reset_index()
result_df = result_df[['way', 'revenue_nuse_As-is', 'revenue_nuse_To-be', 'revenue_nuse_growth']]
result_df

In [ ]:
# Step 2: date 및 date와 way별로 revenue_nuse의 평균 계산
summary = df.groupby(['date', 'way'])['revenue_nuse'].mean().reset_index()
date_totals = df.groupby('date')['revenue_nuse'].mean().reset_index()
summary = summary.merge(date_totals, on='date', suffixes=('', '_total'))

# 'As-is'와 'To-be' 데이터 복사본 생성 및 처리
asis_data = summary[summary['date'] == 'As-is'].copy()
tobe_data = summary[summary['date'] == 'To-be'].copy()

# 'As-is' 데이터의 'revenue_nuse' 값을 음수로 변환
asis_data['revenue_nuse'] = asis_data['revenue_nuse'] * -1

# 피라미드 차트 생성
fig = go.Figure()

# 'As-is' 데이터 추가 (음수값 사용, 정수로 포맷팅)
fig.add_trace(go.Bar(
    y=asis_data['way'],
    x=asis_data['revenue_nuse'],
    name='As-is',
    orientation='h',
    text=[int(value) for value in -asis_data['revenue_nuse']],  # 소수점 제거
    textposition='auto'
))

# 'To-be' 데이터 추가 (정수로 포맷팅)
fig.add_trace(go.Bar(
    y=tobe_data['way'],
    x=tobe_data['revenue_nuse'],
    name='To-be',
    orientation='h',
    text=[int(value) for value in tobe_data['revenue_nuse']],  # 소수점 제거
    textposition='auto'
))

# 동적으로 x축 tickvals과 ticktext 생성
max_value = max(tobe_data['revenue_nuse'].max(), -asis_data['revenue_nuse'].min())
tick_step = int(max_value // 5) or 1
tick_values = list(range(-int(max_value), int(max_value) + 1, tick_step))
tick_texts = [str(abs(i)) for i in tick_values]

# 레이아웃 업데이트
fig.update_layout(
    title='Date별 예약타입별 revenue_nuse (피라미드 차트)',
    barmode='overlay',
    bargap=0.1,
    xaxis=dict(
        title='Revenue per Use',
        tickvals=tick_values,
        ticktext=tick_texts
    ),
    yaxis=dict(
        title='예약타입'
    ),
    legend=dict(title='Date')
)

fig.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import plotly.graph_objects as go

# 데이터 준비 및 정규화 과정
# 전체 summary 데이터에서 'revenue_nuse'의 평균을 계산합니다.
summary = df.groupby(['date', 'way'])['revenue_nuse'].mean().reset_index()

# MinMaxScaler를 사용하여 'revenue_nuse'의 값을 정규화합니다.
scaler = MinMaxScaler()
# 정규화를 위해 'revenue_nuse' 열만 포함하는 DataFrame을 사용합니다.
normalized_values = scaler.fit_transform(summary[['revenue_nuse']])
# 정규화된 결과를 summary에 추가합니다.
summary['revenue_nuse_normalized'] = normalized_values

# 'As-is'와 'To-be' 데이터 복사본 생성 및 정규화된 값으로 업데이트
asis_data = summary[summary['date'] == 'As-is'].copy()
tobe_data = summary[summary['date'] == 'To-be'].copy()

# 피라미드 차트 생성을 위한 데이터 준비
fig = go.Figure()

# 'As-is' 데이터 추가
fig.add_trace(go.Bar(
    y=asis_data['way'],
    x=-asis_data['revenue_nuse_normalized'],  # 음수로 변환하여 'As-is'를 왼쪽으로 표시
    name='As-is',
    orientation='h',
    text=[f"{-value:.2f}" for value in asis_data['revenue_nuse_normalized']],  # 정규화된 값을 음수로 표시
    textposition='auto'
))

# 'To-be' 데이터 추가
fig.add_trace(go.Bar(
    y=tobe_data['way'],
    x=tobe_data['revenue_nuse_normalized'],
    name='To-be',
    orientation='h',
    text=[f"{value:.2f}" for value in tobe_data['revenue_nuse_normalized']],
    textposition='auto'
))

# 차트 레이아웃 설정
fig.update_layout(
    title='Date별 예약타입별 정규화된 Revenue per Use (피라미드 차트)',
    barmode='overlay',
    bargap=0.1,
    xaxis=dict(
        title='Normalized Revenue per Use',
        # 정규화된 값의 범위는 -1에서 1까지이므로, tickvals과 ticktext를 조정할 필요가 있습니다.
        tickvals=[-1, 0, 1],
        ticktext=['1', '0', '1']
    ),
    yaxis=dict(title='예약타입'),
    legend=dict(title='Date')
)

fig.show()

In [ ]:
# Step 2: date 및 date와 way별로 revenue_nuse의 평균 계산
summary = df.groupby(['date', 'way'])['utime_nuse'].mean().reset_index()
date_totals = df.groupby('date')['utime_nuse'].mean().reset_index()
summary = summary.merge(date_totals, on='date', suffixes=('', '_total'))

df_grouped = df.groupby(['way', 'date'])['utime_nuse'].mean().reset_index()

# 'As-is'와 'To-be' 데이터를 wide format으로 변환
df_pivot = df_grouped.pivot(index='way', columns='date', values=['utime_nuse'])

# 컬럼 이름 재정의
df_pivot.columns = [f'{i}_{j}' for i, j in df_pivot.columns]

# 증감률 계산
df_pivot['utime_nuse_growth'] = (df_pivot['utime_nuse_To-be'] - df_pivot['utime_nuse_As-is']) / df_pivot['utime_nuse_As-is'] * 100

# 결과 데이터프레임 재구성
result_df = df_pivot.reset_index()
result_df = result_df[['way', 'utime_nuse_As-is', 'utime_nuse_To-be', 'utime_nuse_growth']]
result_df

In [ ]:
# 'As-is'와 'To-be' 데이터 복사본 생성 및 처리
asis_data = summary[summary['date'] == 'As-is'].copy()
tobe_data = summary[summary['date'] == 'To-be'].copy()

# 'As-is' 데이터의 'revenue_nuse' 값을 음수로 변환
asis_data['utime_nuse'] = asis_data['utime_nuse'] * -1

# 피라미드 차트 생성
fig = go.Figure()

# 'As-is' 데이터 추가 (음수값 사용, 정수로 포맷팅)
fig.add_trace(go.Bar(
    y=asis_data['way'],
    x=asis_data['utime_nuse'],
    name='As-is',
    orientation='h',
    text=[int(value) for value in -asis_data['utime_nuse']],  # 소수점 제거
    textposition='auto'
))

# 'To-be' 데이터 추가 (정수로 포맷팅)
fig.add_trace(go.Bar(
    y=tobe_data['way'],
    x=tobe_data['utime_nuse'],
    name='To-be',
    orientation='h',
    text=[int(value) for value in tobe_data['utime_nuse']],  # 소수점 제거
    textposition='auto'
))

# 동적으로 x축 tickvals과 ticktext 생성
max_value = max(tobe_data['utime_nuse'].max(), -asis_data['utime_nuse'].min())
tick_step = int(max_value // 5) or 1
tick_values = list(range(-int(max_value), int(max_value) + 1, tick_step))
tick_texts = [str(abs(i)) for i in tick_values]

# 레이아웃 업데이트
fig.update_layout(
    title='Date별 예약타입별 utime_nuse (피라미드 차트)',
    barmode='overlay',
    bargap=0.1,
    xaxis=dict(
        title='Utime per Use',
        tickvals=tick_values,
        ticktext=tick_texts
    ),
    yaxis=dict(
        title='예약타입'
    ),
    legend=dict(title='Date')
)

fig.show()

In [ ]:
# MinMaxScaler를 사용하여 'utime_nuse'의 값을 정규화합니다.
scaler = MinMaxScaler()
# 전체 'summary'에 대해 'utime_nuse' 열만을 대상으로 정규화를 적용하고, 그 결과를 새 열에 저장합니다.
summary['utime_nuse_normalized'] = scaler.fit_transform(summary[['utime_nuse']])

# 'As-is'와 'To-be' 데이터 복사본 생성 및 정규화된 값 할당
asis_data = summary[summary['date'] == 'As-is'].copy()
tobe_data = summary[summary['date'] == 'To-be'].copy()

# 'As-is' 데이터의 정규화된 'utime_nuse' 값을 음수로 변환
asis_data['utime_nuse_normalized'] = asis_data['utime_nuse_normalized'] * -1

# 피라미드 차트 생성
fig = go.Figure()

# 'As-is' 데이터 추가 (음수 정규화된 값 사용)
fig.add_trace(go.Bar(
    y=asis_data['way'],
    x=asis_data['utime_nuse_normalized'],
    name='As-is',
    orientation='h',
    text=[f"{value:.2f}" for value in -asis_data['utime_nuse_normalized']],  # 소수점 두 자리로 표시
    textposition='auto'
))

# 'To-be' 데이터 추가 (정규화된 값 사용)
fig.add_trace(go.Bar(
    y=tobe_data['way'],
    x=tobe_data['utime_nuse_normalized'],
    name='To-be',
    orientation='h',
    text=[f"{value:.2f}" for value in tobe_data['utime_nuse_normalized']],  # 소수점 두 자리로 표시
    textposition='auto'
))

# 동적으로 x축 tickvals과 ticktext 생성 (정규화된 값의 범위 고려)
max_abs_value = max(abs(asis_data['utime_nuse_normalized'].min()), abs(tobe_data['utime_nuse_normalized'].max()))
tick_values = [-max_abs_value, 0, max_abs_value]
tick_texts = [f"{-max_abs_value:.2f}", "0", f"{max_abs_value:.2f}"]

# 레이아웃 업데이트
fig.update_layout(
    title='Date별 예약타입별 정규화된 Utime per Use (피라미드 차트)',
    barmode='overlay',
    bargap=0.1,
    xaxis=dict(
        title='Normalized Utime per Use',
        tickvals=tick_values,
        ticktext=tick_texts
    ),
    yaxis=dict(
        title='예약타입'
    ),
    legend=dict(title='Date')
)

fig.show()

In [ ]:
# Step 2: date 및 date와 way별로 revenue_nuse의 평균 계산
summary = df.groupby(['date', 'way'])['margin_nuse'].mean().reset_index()
date_totals = df.groupby('date')['margin_nuse'].mean().reset_index()
summary = summary.merge(date_totals, on='date', suffixes=('', '_total'))

df_grouped = df.groupby(['way', 'date'])['margin_nuse'].mean().reset_index()

# 'As-is'와 'To-be' 데이터를 wide format으로 변환
df_pivot = df_grouped.pivot(index='way', columns='date', values=['margin_nuse'])

# 컬럼 이름 재정의
df_pivot.columns = [f'{i}_{j}' for i, j in df_pivot.columns]

# 증감률 계산
df_pivot['margin_nuse_growth'] = (df_pivot['margin_nuse_To-be'] - df_pivot['margin_nuse_As-is']) / df_pivot['margin_nuse_As-is'] * 100

# 결과 데이터프레임 재구성
result_df = df_pivot.reset_index()
result_df = result_df[['way', 'margin_nuse_As-is', 'margin_nuse_To-be', 'margin_nuse_growth']]
result_df

In [ ]:
# 'As-is'와 'To-be' 데이터 복사본 생성 및 처리
asis_data = summary[summary['date'] == 'As-is'].copy()
tobe_data = summary[summary['date'] == 'To-be'].copy()

# 'As-is' 데이터의 'revenue_nuse' 값을 음수로 변환
asis_data['margin_nuse'] = asis_data['margin_nuse'] * -1

# 피라미드 차트 생성
fig = go.Figure()

# 'As-is' 데이터 추가 (음수값 사용, 정수로 포맷팅)
fig.add_trace(go.Bar(
    y=asis_data['way'],
    x=asis_data['margin_nuse'],
    name='As-is',
    orientation='h',
    text=[int(value) for value in -asis_data['margin_nuse']],  # 소수점 제거
    textposition='auto'
))

# 'To-be' 데이터 추가 (정수로 포맷팅)
fig.add_trace(go.Bar(
    y=tobe_data['way'],
    x=tobe_data['margin_nuse'],
    name='To-be',
    orientation='h',
    text=[int(value) for value in tobe_data['margin_nuse']],  # 소수점 제거
    textposition='auto'
))

# 동적으로 x축 tickvals과 ticktext 생성
max_value = max(tobe_data['margin_nuse'].max(), -asis_data['margin_nuse'].min())
tick_step = int(max_value // 5) or 1
tick_values = list(range(-int(max_value), int(max_value) + 1, tick_step))
tick_texts = [str(abs(i)) for i in tick_values]

# 레이아웃 업데이트
fig.update_layout(
    title='Date별 예약타입별 margin_nuse (피라미드 차트)',
    barmode='overlay',
    bargap=0.1,
    xaxis=dict(
        title='Margin per Use',
        tickvals=tick_values,
        ticktext=tick_texts
    ),
    yaxis=dict(
        title='예약타입'
    ),
    legend=dict(title='Date')
)

fig.show()

In [ ]:
# MinMaxScaler를 사용하여 'utime_nuse'의 값을 정규화합니다.
scaler = MinMaxScaler()
# 전체 'summary'에 대해 'utime_nuse' 열만을 대상으로 정규화를 적용하고, 그 결과를 새 열에 저장합니다.
summary['margin_nuse_normalized'] = scaler.fit_transform(summary[['margin_nuse']])

# 'As-is'와 'To-be' 데이터 복사본 생성 및 정규화된 값 할당
asis_data = summary[summary['date'] == 'As-is'].copy()
tobe_data = summary[summary['date'] == 'To-be'].copy()

# 'As-is' 데이터의 정규화된 'utime_nuse' 값을 음수로 변환
asis_data['margin_nuse_normalized'] = asis_data['margin_nuse_normalized'] * -1

# 피라미드 차트 생성
fig = go.Figure()

# 'As-is' 데이터 추가 (음수 정규화된 값 사용)
fig.add_trace(go.Bar(
    y=asis_data['way'],
    x=asis_data['margin_nuse_normalized'],
    name='As-is',
    orientation='h',
    text=[f"{value:.2f}" for value in -asis_data['margin_nuse_normalized']],  # 소수점 두 자리로 표시
    textposition='auto'
))

# 'To-be' 데이터 추가 (정규화된 값 사용)
fig.add_trace(go.Bar(
    y=tobe_data['way'],
    x=tobe_data['margin_nuse_normalized'],
    name='To-be',
    orientation='h',
    text=[f"{value:.2f}" for value in tobe_data['margin_nuse_normalized']],  # 소수점 두 자리로 표시
    textposition='auto'
))

# 동적으로 x축 tickvals과 ticktext 생성 (정규화된 값의 범위 고려)
max_abs_value = max(abs(asis_data['margin_nuse_normalized'].min()), abs(tobe_data['margin_nuse_normalized'].max()))
tick_values = [-max_abs_value, 0, max_abs_value]
tick_texts = [f"{-max_abs_value:.2f}", "0", f"{max_abs_value:.2f}"]

# 레이아웃 업데이트
fig.update_layout(
    title='Date별 예약타입별 정규화된 Margin per Use (피라미드 차트)',
    barmode='overlay',
    bargap=0.1,
    xaxis=dict(
        title='Normalized Margin per Use',
        tickvals=tick_values,
        ticktext=tick_texts
    ),
    yaxis=dict(
        title='예약타입'
    ),
    legend=dict(title='Date')
)

fig.show()

## 📈 *부름 비용의 확대*
- 부름 배반비용
- 건당 부름 배반비용

기존과 거의 비슷함
- 부름 왕복에서 감소된 비용이 부름 편도로 증가

In [ ]:
df_d2d = df[df['way'].isin(['d2d_round', 'd2d_onway'])]

# Step 2: date 및 date와 way별로 revenue_nuse의 평균 계산
summary = df.groupby(['date', 'way'])['d2d_cost'].mean().reset_index()
date_totals = df.groupby('date')['d2d_cost'].mean().reset_index()
summary = summary.merge(date_totals, on='date', suffixes=('', '_total'))

summary_grouped = summary.groupby(['way', 'date'])['d2d_cost'].sum().reset_index()

# 'As-is'와 'To-be' 데이터를 wide format으로 변환
df_pivot = summary_grouped.pivot(index='way', columns='date', values=['d2d_cost'])

# 컬럼 이름 재정의
df_pivot.columns = [f'{i}_{j}' for i, j in df_pivot.columns]

# 증감률 계산
df_pivot['d2d_cost_growth'] = (df_pivot['d2d_cost_To-be'] - df_pivot['d2d_cost_As-is']) / df_pivot['d2d_cost_As-is'] * 100

# 결과 데이터프레임 재구성
result_df = df_pivot.reset_index()
result_df = result_df[['way', 'd2d_cost_As-is', 'd2d_cost_To-be', 'd2d_cost_growth']]
result_df

In [ ]:
# Step 3: 바 차트로 결과 시각화, 여기서는 revenue_nuse의 평균을 사용
fig = px.bar(summary, x='way', y='d2d_cost', color='date',
             title='Date별 예약타입별 d2d_cost',
             barmode='group',
             text='d2d_cost',
             labels={'d2d_cost': '부름 배반비용', 'way': '예약타입', 'date': 'Date'})

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')  # 소수점 두 자리로 포맷
fig.show()

In [ ]:
# date 및 date와 way별로 d2d_cost의 평균 계산
summary = df.groupby(['date', 'way'])['d2d_cost'].mean().reset_index()

# MinMaxScaler 인스턴스 생성
scaler = MinMaxScaler()

# 정규화를 위한 함수 정의
def normalize_group(x):
    return scaler.fit_transform(x.values.reshape(-1, 1)).flatten()

# 각 date 별로 정규화 수행
summary['d2d_cost_normalized'] = summary.groupby('date')['d2d_cost'].transform(normalize_group)

# 바 차트로 결과 시각화, 정규화된 d2d_cost 평균을 사용
fig = px.bar(summary, x='way', y='d2d_cost_normalized', color='date',
             title='Date별 예약타입별 정규화된 d2d_cost 평균',
             barmode='group',
             text='d2d_cost_normalized',  # 정규화된 d2d_cost 평균 표시
             labels={'d2d_cost_normalized': '정규화된 부름 배반비용', 'way': '예약타입', 'date': 'Date'})

# 정규화된 값으로 텍스트 포맷 지정
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')

fig.show()

## 📈 *부름 편도 오픈/지역 확대 전/후 유휴율 비교*
- 지역 확대, 편도 오픈후 유휴율 감소세   
   ```23.12.22 폭설 이슈로 급증했던 점 참고```   

✔ 유휴율 개선에 효과적

In [ ]:
query_idle = """
select
  date,
  CASE WHEN date BETWEEN '2023-11-06' AND '2023-12-07' THEN 'Before'
       WHEN date BETWEEN '2023-12-08' AND '2024-01-08' THEN 'After' END as date_type,

  round(sum(dp_min/(1440)),0) as total_car_cnt,
  count(case when dp_min >0 and op_min = 0 then car_id end) as idle,
  sum(op_min)/sum(dp_min) as op_rate,
  count(case when dp_min >0 and op_min = 0 then car_id end)/round(sum(dp_min/(1440)),0) as idle_rate,
  sum(dp_min)/60 as dp_hour,
  sum(op_min)/60 as op_hour,
  sum(bl_min)/60 as bl_hour,
  sum(op_min_passport_subscription)/60 as p_hour

from `socar-data.socar_biz.operation_per_car_daily_v2`
where region1 in ("제주특별자치도")
and date >= '2023-11-06' and date <= "2024-01-08"
and sharing_type in ('socar','zplus')
and zone_id not in (122,2184,11480,12072,12097,10736,10738,11947,13787,13858,14494)
group by date, date_type
"""

df_idle = pd.io.gbq.read_gbq(
    query=query_idle,
    project_id="socar-data"
)

In [ ]:
# Calculate average idle_rate for each date_type
avg_idle_rate = df_idle.groupby(['date', 'date_type'])['idle_rate'].mean().reset_index()

# 전/후 유휴율과 부름 건수 평균 분포도 as a line graph
fig = px.line(avg_idle_rate, x='date', y='idle_rate', color='date_type',
              title='전/후 유휴율 추세선',
              labels={'idle_rate': '제주 전체 평균 유휴율'})

# y축의 범위를 0.0에서 0.2으로 설정
fig.update_layout(yaxis=dict(range=[0.0, 0.3]))

fig.show()

## 📈 *부름 편도 오픈/지역 확대 전/후 핸들 운행률 비교*
- 지역 확대, 편도 오픈후 핸들 운행률 유지
   ```지역 확대와 편도 오픈에 따라 배반 지연은 없음```   

✔ 차량 배반에 문제 없음

In [ ]:
query = """
  WITH base AS (
    SELECT
      r.id AS rid, /* 옥스트라 예약번호 */
      r.state, /* 핸들의 상태값 0:취소, 3:완료 */
      hh.userId AS handler_num, /* 핸들러 회원번호 */
      CASE WHEN r.member_id = 3806971 THEN '핸들러'
          WHEN r.member_id = 52846 THEN '탁송업체' END AS member_type, /* 핸들러/탁송업체 운행 구분 */

      CASE WHEN r.way = 'd2d_export_h' THEN '부름(배차)'
          WHEN r.way = 'd2d_import_h' THEN '부름(반차)'
          WHEN r.way = 'd2d_oneway' THEN '부름편도'
          WHEN r.way = 'd2d_rev' THEN '부름(진행전)'
          WHEN r.way = 'd2d_round' THEN '부름왕복'
          WHEN r.way = 'handle' THEN '핸들'
          WHEN r.way = 'oneway' THEN '편도'
          WHEN r.way = 'oneway_float' THEN '편도(플로팅)'
          WHEN r.way = 'return' THEN '회송'
          WHEN r.way = 'return_h' THEN '회송(핸들)'
          WHEN r.way = 'round' THEN '왕복'
          WHEN r.way = 'z2d_oneway' THEN '존편도' END as way,

      date(r.start_at, "Asia/Seoul") as date,
      date(ifnull(r.reserved_end_at, r.end_at), "Asia/Seoul") as return_at,

      con.memo as memo, /* 핸들러 미매칭으로 탁송전환 된 건인지 확인 용도 */

      cz.address /* 쏘카존의 주소, region으로 구분하기 어려운 지역 구분 용도 */
      ,r.way as parent_way,
      timestamp_diff(hr.onboardat, hr.openat, minute) as dur

    FROM `socar-data.tianjin_replica.reservation_info` AS r
    LEFT JOIN `socar-handler.handler_replica.socarhandler_reservation` AS hr /* 핸들러 예약 테이블 (핸들러 배정된 내역만 적재) */
    ON hr.scReservationId = r.id
    LEFT JOIN `socar-handler.handler_replica.socarhandler_handle` AS hh /* 핸들별 추가 정보용 테이블 (핸들러 배정된 내역만 적재) */
    ON hh.reservationId = hr.reservationId
    LEFT JOIN `socar-data.tianjin_replica.carzone_info` AS cz
    ON cz.id = r.zone_id
    LEFT JOIN (SELECT *
                FROM `socar-data.tianjin_replica.reservation_memo`
                WHERE memo LIKE '%판매대기 제한시간%') AS con /* 옥스트라 예약메모 추출 (특정 문구가 포함된 건들을 탁송업체 전환이라고 보기 위함) */
    ON con.info_key = r.id
    WHERE r.way in ('d2d_export_h','d2d_import_h','handle') /* d2d_export_h 반차: d2d_import_h 이벤트핸들: handle */
    AND r.state in (3,0) /* 완료&취소 핸들 집계 */
    AND DATETIME(r.start_at,'Asia/Seoul') >= '2022-01-01'
    AND (r.member_id = 3806971 OR (r.member_id = 52846 AND con.memo IS NOT NULL))
    AND cz.region1 = '제주특별자치도'
  ),

  base2 AS (
    SELECT
      date(start_at, 'Asia/Seoul') as sdate,
      way,
      r.id as rid,
      datetime_diff(end_at, start_at, hour) as dur

    FROM `tianjin_replica.reservation_info` r LEFT JOIN `tianjin_replica.carzone_info` z ON r.zone_id = z.id
    WHERE z.region1 = '제주특별자치도' AND r.member_imaginary IN (0, 9)
                                    AND r.state IN (1, 2, 3) /* 1 예약 2 운행 3 완료 */
                                    AND r.way IN ('d2d_round', 'd2d_oneway')
                                    AND date(start_at, 'Asia/Seoul') >= '2023-11-06'
                                    AND date(start_at, 'Asia/Seoul') <= "2024-01-08"
  ),

  d2d_calc AS (
    SELECT
      sdate,
      count(rid) as rev_cnt,
      count(CASE WHEN way = 'd2d_round' THEN rid END) as round_rev,
      count(CASE WHEN way = 'd2d_oneway' THEN rid END) as oneway_rev,
      sum(dur) as dur,
      sum(dur)/count(rid) as rev_dur,
      sum(CASE WHEN way = 'd2d_round' THEN dur END) /count(CASE WHEN way = 'd2d_round' THEN rid END) as round_rev_dur,
      sum(CASE WHEN way = 'd2d_oneway' THEN dur END) /count(CASE WHEN way = 'd2d_oneway' THEN rid END) as oneway_rev_dur
    FROM base2
    GROUP BY sdate
  ),

  handle_calc AS (
    SELECT
      date,
      count(CASE WHEN way LIKE '%부름%' THEN rid END) as rev_total,
      count(CASE WHEN member_type = '핸들러' AND way LIKE "%부름%" THEN rid END) as handle_cnt,
      count(CASE WHEN member_type = '탁송업체' AND way LIKE "%부름%" THEN rid END) as local_cnt,
      avg(dur) as d_minute,

    FROM base
    GROUP BY date
  )

  SELECT

    dc.sdate,

    CASE WHEN date >= '2023-11-06' AND date <= '2023-12-07' THEN 'As-is'
        WHEN date >= '2023-12-08' AND date <= '2024-01-08' THEN 'To-be' END as date_type,

    dc.rev_cnt,
    dc.round_rev,
    dc.oneway_rev,
    round(dc.dur/dc.rev_cnt, 0) as rev_dur,
    dc.round_rev_dur,
    dc.oneway_rev_dur,

    hc.rev_total,
    hc.handle_cnt,
    hc.local_cnt,
    round(hc.d_minute, 0) as d_minute,
    round(hc.handle_cnt/hc.rev_total, 2) as hanlde_rate

  FROM d2d_calc dc LEFT JOIN handle_calc hc ON dc.sdate = hc.date
  ORDER BY sdate desc
  """

df_handle = pd.io.gbq.read_gbq(
    query=query,
    project_id="socar-data"
)

df_handle.info()

In [ ]:
avg_handle = df_handle.groupby('date_type')['hanlde_rate'].mean().reset_index()
avg_handle

In [ ]:
# Calculate average idle_rate for each date_type
avg_handle_rate = df_handle.groupby(['sdate', 'date_type'])['hanlde_rate'].mean().reset_index()

# 전/후 유휴율과 부름 건수 평균 분포도 as a line graph
fig = px.line(avg_handle_rate, x='sdate', y='hanlde_rate', color='date_type',
              title='전/후 핸들 운행률 추세선',
              labels={'hanlde_rate': '제주 전체 평균 핸들 운행률'})

fig.update_layout(yaxis=dict(range=[0.4, 1.5]))

fig.show()

## ✅ 서비스 확대후 매출과 이용 추이
- 지역 확대, 편도 오픈후 핸들 운행률 유지
   ```지역 확대와 편도 오픈에 따라 배반 지연은 없음```   

✔ 차량 배반에 문제 없음

In [ ]:
df.info()

In [ ]:
# 'date'와 'way' 별로 지표들의 평균을 계산
# 이 예제에서는 sum()을 사용
grouped_df = df.groupby(['date', 'way']).sum().reset_index()

# 'date'를 컬럼으로 변환하여 각 지표별로 'As-is'와 'To-be' 상태를 분리
pivot_df = grouped_df.pivot(index='way', columns='date')

# 컬럼 이름 재정의
pivot_df.columns = [f'{col[0]}_{col[1]}' for col in pivot_df.columns]

# 증감 계산
for metric in ['nuse', 'utime', 'revenue', 'margin', 'd2d_cost']:
    pivot_df[f'{metric}_change'] = pivot_df[f'{metric}_To-be'] - pivot_df[f'{metric}_As-is']

# 증감 비교 표를 확인
print(pivot_df.reset_index()[['way'] + [f'{metric}_change' for metric in ['nuse', 'utime', 'revenue', 'margin', 'd2d_cost']]])
pivot_df

In [ ]:
from plotly.subplots import make_subplots

# 1행 5열의 서브플롯 구성을 생성
fig = make_subplots(rows=1, cols=5, subplot_titles=('Revenue Change', 'Margin Change', 'Nuse Change', 'Utime Change', 'D2d Cost Change'))

# 각 지표별로 서브플롯에 바 차트 추가
metrics = ['revenue', 'margin', 'nuse', 'utime', 'd2d_cost']
for i, metric in enumerate(metrics, start=1):
    fig.add_trace(
        go.Bar(x=pivot_df.index, y=pivot_df[f'{metric}_change'], name=f'{metric.capitalize()} Change'),
        row=1, col=i
    )

# 레이아웃 업데이트
fig.update_layout(height=600, width=1200, title_text="Change from As-is to To-be across Metrics", showlegend=False)
fig.update_layout(barmode='group')

# x축 및 y축 레이블 업데이트
fig.update_xaxes(title_text="Reservation Way", row=1, col=1)
fig.update_yaxes(title_text="Change", row=1, col=1)

# 각 서브플롯에 대해 축 및 레이아웃 조정이 필요한 경우 추가적으로 업데이트

fig.show()

In [ ]:
# 'date'와 'way' 별로 지표들의 합계를 계산하면서, numeric_only=True를 명시적으로 지정
grouped_df = df.groupby(['date', 'way']).sum(numeric_only=True).reset_index()

# pivot 대신 pivot_table 사용하고 fill_value=0 지정
pivot_df = grouped_df.pivot_table(index='way', columns='date', values=['nuse', 'utime', 'revenue', 'margin', 'd2d_cost'], fill_value=0)

# 컬럼 이름 재정의
pivot_df.columns = [f'{col[0]}_{col[1]}' for col in pivot_df.columns]

# 증감 계산 (As-is 값이 0일 경우 To-be 값으로 계산)
for metric in ['nuse', 'utime', 'revenue', 'margin', 'd2d_cost']:
    as_is_col = f'{metric}_As-is'
    to_be_col = f'{metric}_To-be'
    pivot_df[f'{metric}_change'] = pivot_df[to_be_col] - pivot_df[as_is_col]

# 제주 서브플롯
fig = make_subplots(rows=1, cols=5, subplot_titles=('Revenue Change', 'Margin Change', 'Nuse Change', 'Utime Change', 'D2d Cost Change'))

# 각 지표별로 서브플롯에 바 차트 추가
metrics = ['revenue', 'margin', 'nuse', 'utime', 'd2d_cost']
for i, metric in enumerate(metrics, start=1):
    fig.add_trace(
        go.Bar(x=pivot_df.index, y=pivot_df[f'{metric}_change'], name=f'{metric.capitalize()} Change'),
        row=1, col=i
    )

# 레이아웃 업데이트
fig.update_layout(height=600, width=1200, title_text="Change from As-is to To-be across Metrics", showlegend=False)
fig.update_layout(barmode='group')

# x축 및 y축 레이블 업데이트
fig.update_xaxes(title_text="Reservation Way", row=1, col=1)
fig.update_yaxes(title_text="Change", row=1, col=1)

fig.show()

In [ ]:
# 제주시
df_jeju = df[df['region2'] == '제주시']

# 'date'와 'way' 별로 지표들의 평균을 계산
grouped_df = df_jeju.groupby(['date', 'way']).sum().reset_index()

# 'date'를 컬럼으로 변환하여 각 지표별로 'As-is'와 'To-be' 상태를 분리
pivot_df = grouped_df.pivot(index='way', columns='date')

# 컬럼 이름 재정의
pivot_df.columns = [f'{col[0]}_{col[1]}' for col in pivot_df.columns]

# 증감 계산
for metric in ['nuse', 'utime', 'revenue', 'margin', 'd2d_cost']:
    pivot_df[f'{metric}_change'] = pivot_df[f'{metric}_To-be'] - pivot_df[f'{metric}_As-is']

In [ ]:
# 제주 서브플랏
fig = make_subplots(rows=1, cols=5, subplot_titles=('Revenue Change', 'Margin Change', 'Nuse Change', 'Utime Change', 'D2d Cost Change'))

# 각 지표별로 서브플롯에 바 차트 추가
metrics = ['revenue', 'margin', 'nuse', 'utime', 'd2d_cost']
for i, metric in enumerate(metrics, start=1):
    fig.add_trace(
        go.Bar(x=pivot_df.index, y=pivot_df[f'{metric}_change'], name=f'{metric.capitalize()} Change'),
        row=1, col=i
    )

# 레이아웃 업데이트
fig.update_layout(height=600, width=1200, title_text="Jeju-Si Change from As-is to To-be across Metrics", showlegend=False)
fig.update_layout(barmode='group')

# x축 및 y축 레이블 업데이트
fig.update_xaxes(title_text="Reservation Way", row=1, col=1)
fig.update_yaxes(title_text="Change", row=1, col=1)

# 각 서브플롯에 대해 축 및 레이아웃 조정이 필요한 경우 추가적으로 업데이트
fig.show()

In [ ]:
# 제주시 데이터 필터링
df_jeju = df[df['region2'] == '제주시']

# 'date'와 'way' 별로 지표들의 합계를 계산하면서, numeric_only=True를 명시적으로 지정
grouped_df = df_jeju.groupby(['date', 'way']).sum(numeric_only=True).reset_index()

# pivot 대신 pivot_table 사용하고 fill_value=0 지정
pivot_df = grouped_df.pivot_table(index='way', columns='date', values=['nuse', 'utime', 'revenue', 'margin', 'd2d_cost'], fill_value=0)

# 컬럼 이름 재정의
pivot_df.columns = [f'{col[0]}_{col[1]}' for col in pivot_df.columns]

# 증감 계산 (As-is 값이 0일 경우 To-be 값으로 계산)
for metric in ['nuse', 'utime', 'revenue', 'margin', 'd2d_cost']:
    as_is_col = f'{metric}_As-is'
    to_be_col = f'{metric}_To-be'
    pivot_df[f'{metric}_change'] = pivot_df[to_be_col] - pivot_df[as_is_col]

# 제주 서브플롯
fig = make_subplots(rows=1, cols=5, subplot_titles=('Revenue Change', 'Margin Change', 'Nuse Change', 'Utime Change', 'D2d Cost Change'))

# 각 지표별로 서브플롯에 바 차트 추가
metrics = ['revenue', 'margin', 'nuse', 'utime', 'd2d_cost']
for i, metric in enumerate(metrics, start=1):
    fig.add_trace(
        go.Bar(x=pivot_df.index, y=pivot_df[f'{metric}_change'], name=f'{metric.capitalize()} Change'),
        row=1, col=i
    )

# 레이아웃 업데이트
fig.update_layout(height=600, width=1200, title_text="Jeju-Si Change from As-is to To-be across Metrics", showlegend=False)
fig.update_layout(barmode='group')

# x축 및 y축 레이블 업데이트
fig.update_xaxes(title_text="Reservation Way", row=1, col=1)
fig.update_yaxes(title_text="Change", row=1, col=1)

fig.show()

In [ ]:
# 서귀포시
df_swo = df[df['region2'] == '서귀포시']

# 'date'와 'way' 별로 지표들의 평균을 계산
grouped_df = df_swo.groupby(['date', 'way']).sum().reset_index()

# 'date'를 컬럼으로 변환하여 각 지표별로 'As-is'와 'To-be' 상태를 분리
pivot_df = grouped_df.pivot(index='way', columns='date')

# 컬럼 이름 재정의
pivot_df.columns = [f'{col[0]}_{col[1]}' for col in pivot_df.columns]

# 증감 계산
for metric in ['nuse', 'utime', 'revenue', 'margin', 'd2d_cost']:
    pivot_df[f'{metric}_change'] = pivot_df[f'{metric}_To-be'] - pivot_df[f'{metric}_As-is']

In [ ]:
# 서귀포 서브플랏
fig = make_subplots(rows=1, cols=5, subplot_titles=('Revenue Change', 'Margin Change', 'Nuse Change', 'Utime Change', 'D2d Cost Change'))

# 각 지표별로 서브플롯에 바 차트 추가
metrics = ['revenue', 'margin', 'nuse', 'utime', 'd2d_cost']
for i, metric in enumerate(metrics, start=1):
    fig.add_trace(
        go.Bar(x=pivot_df.index, y=pivot_df[f'{metric}_change'], name=f'{metric.capitalize()} Change'),
        row=1, col=i
    )

# 레이아웃 업데이트
fig.update_layout(height=600, width=1200, title_text="Seogwipo-Si Change from As-is to To-be across Metrics", showlegend=False)
fig.update_layout(barmode='group')

# x축 및 y축 레이블 업데이트
fig.update_xaxes(title_text="Reservation Way", row=1, col=1)
fig.update_yaxes(title_text="Change", row=1, col=1)

# 각 서브플롯에 대해 축 및 레이아웃 조정이 필요한 경우 추가적으로 업데이트
fig.show()

In [ ]:
# 서귀포시 필터링
df_swo = df[df['region2'] == '서귀포시']

# 'date'와 'way' 별로 지표들의 합계를 계산하면서, numeric_only=True를 명시적으로 지정
grouped_df = df_swo.groupby(['date', 'way']).sum(numeric_only=True).reset_index()

# pivot 대신 pivot_table 사용하고 fill_value=0 지정
pivot_df = grouped_df.pivot_table(index='way', columns='date', values=['nuse', 'utime', 'revenue', 'margin', 'd2d_cost'], fill_value=0)

# 컬럼 이름 재정의
pivot_df.columns = [f'{col[0]}_{col[1]}' for col in pivot_df.columns]

# 증감 계산 (As-is 값이 0일 경우 To-be 값으로 계산)
for metric in ['nuse', 'utime', 'revenue', 'margin', 'd2d_cost']:
    as_is_col = f'{metric}_As-is'
    to_be_col = f'{metric}_To-be'
    pivot_df[f'{metric}_change'] = pivot_df[to_be_col] - pivot_df[as_is_col]

# 제주 서브플롯
fig = make_subplots(rows=1, cols=5, subplot_titles=('Revenue Change', 'Margin Change', 'Nuse Change', 'Utime Change', 'D2d Cost Change'))

# 각 지표별로 서브플롯에 바 차트 추가
metrics = ['revenue', 'margin', 'nuse', 'utime', 'd2d_cost']
for i, metric in enumerate(metrics, start=1):
    fig.add_trace(
        go.Bar(x=pivot_df.index, y=pivot_df[f'{metric}_change'], name=f'{metric.capitalize()} Change'),
        row=1, col=i
    )

# 레이아웃 업데이트
fig.update_layout(height=600, width=1200, title_text="Seogwipo-Si Change from As-is to To-be across Metrics", showlegend=False)
fig.update_layout(barmode='group')

# x축 및 y축 레이블 업데이트
fig.update_xaxes(title_text="Reservation Way", row=1, col=1)
fig.update_yaxes(title_text="Change", row=1, col=1)

fig.show()

# 2. 부름 확대 전/후의 부름을 호출한 예약지 위치 변화


In [ ]:
query = """
  SELECT
      CASE WHEN date >= '2023-11-06' AND date <= '2023-12-07' THEN 'As-is'
           WHEN date >= '2023-12-08' AND date <= '2024-01-08' THEN 'To-be' END as date,

      scParentId as rid,
      r.way,

      -- hr.startlng, hr.startlat,
      rv.dtod_start_lng as start_lng,
      rv.dtod_start_lat as start_lat,
      CASE WHEN hr.startaddress1 LIKE '%제주시%' THEN  '제주시'
            WHEN hr.startaddress1 LIKE '%서귀포시%' THEN  '서귀포시' END as start_city,

      -- hr.endlng, hr.endlat,
      rv.dtod_end_lng as end_lng,
      rv.dtod_end_lat as end_lat,
      CASE WHEN hr.endAddress1 LIKE '%제주시%' THEN  '제주시'
            WHEN hr.endAddress1 LIKE '%서귀포시%' THEN  '서귀포시' END as end_city,

      rv.reservation_created_lat as c_lat,
      rv.reservation_created_lng as c_lng,

  FROM `socar-data.handler_replica.socarhandler_reservation` hr
  LEFT JOIN `tianjin_replica.reservation_info` r ON hr.scParentId = r.id
  LEFT JOIN `tianjin_replica.carzone_info` z ON r.zone_id = z.id
  LEFT JOIN `soda_store.reservation_v2` rv ON hr.scParentId = rv.reservation_id
  WHERE TIMESTAMP_TRUNC(created, DAY) BETWEEN '2023-11-06' AND '2024-01-08'
  AND z.region1 = '제주특별자치도'
  AND hr.way IN ('d2d_export_h')
  AND r.state IN (1, 2, 3)
  AND dtod_start_lng is not null
  AND dtod_end_lng is not null
  AND reservation_created_lat is not null
  AND reservation_created_lng is not null
  AND r.member_imaginary IN (0, 9)
  """

df = pd.io.gbq.read_gbq(
    query=query,
    project_id="socar-data"
)

df.info()

In [ ]:
df['date'].unique()

In [ ]:
import folium
# 'as-is'와 'to-be'에 대한 색상 직접 매핑
color_map = {
    'As-is': 'blue',
    'To-be': 'red'
}

In [ ]:
# 지도의 초기 위치와 줌 설정하여 기본 지도 생성
map = folium.Map(location=[df['start_lat'].mean(), df['start_lng'].mean()], zoom_start=10)

# 데이터프레임을 순회하며 마커 추가
for _, row in df.dropna(subset=['date']).iterrows():
    # color_map에서 해당 date에 대한 색상을 가져옵니다.
    marker_color = color_map[row['date']]
    folium.Marker(
        location=[row['start_lat'], row['start_lng']],
        popup=f"{row['date']}: {row['start_city']}",
        icon=folium.Icon(color=marker_color)
    ).add_to(map)

# 지도 표시
map

In [ ]:
from folium import plugins

# 'As-is' 데이터에 대한 지도
as_is_df = df[df['date'] == 'As-is']
as_is_map = folium.Map(location=[as_is_df['start_lat'].mean(), as_is_df['start_lng'].mean()], zoom_start=10)

# 'As-is' 데이터에 대한 클러스터 생성 및 마커 추가
as_is_cluster = plugins.MarkerCluster().add_to(as_is_map)
for _, row in as_is_df.iterrows():
    folium.Marker(
        location=[row['start_lat'], row['start_lng']],
        popup=f"As-is: {row['start_city']}",
        icon=folium.Icon(color='blue')
    ).add_to(as_is_cluster)

# 'To-be' 데이터에 대한 지도
to_be_df = df[df['date'] == 'To-be']
to_be_map = folium.Map(location=[to_be_df['start_lat'].mean(), to_be_df['start_lng'].mean()], zoom_start=10)

# 'To-be' 데이터에 대한 클러스터 생성 및 마커 추가
to_be_cluster = plugins.MarkerCluster().add_to(to_be_map)
for _, row in to_be_df.iterrows():
    folium.Marker(
        location=[row['start_lat'], row['start_lng']],
        popup=f"To-be: {row['start_city']}",
        icon=folium.Icon(color='green')
    ).add_to(to_be_cluster)

In [ ]:
# 지도 표시는 각각 별도로 진행해야 합니다. 예를 들어:
as_is_map  # 'As-is' 지도를 Jupyter Notebook에서 표시

In [ ]:
to_be_map  # 'To-be' 지도를 Jupyter Notebook에서 표시

In [ ]:
# 지도의 초기 위치와 줌 설정하여 기본 지도 생성
map = folium.Map(location=[df['c_lat'].mean(), df['c_lng'].mean()], zoom_start=10)

# 데이터프레임을 순회하며 마커 추가
for _, row in df.dropna(subset=['date']).iterrows():
    # color_map에서 해당 date에 대한 색상을 가져옵니다.
    marker_color = color_map[row['date']]
    folium.Marker(
        location=[row['c_lat'], row['c_lng']],
        popup=f"{row['date']}: {row['start_city']}",
        icon=folium.Icon(color=marker_color)
    ).add_to(map)

# 지도 표시
map